In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
from pymongo import MongoClient
import json

In [2]:
def read_xml_to_df(file_path):
    # Parse XML data and get root element
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    data = []
    # find the row element which is nested inside another row element
    for outer_row_element in root.findall('.//row'):
        row_data = {}
        for inner_row_element in outer_row_element.findall('.//row'):
            for child_element in inner_row_element:
                row_data[child_element.tag] = child_element.text
            data.append(row_data.copy())

    # Convert the data into dataframe
    return pd.DataFrame(data)



In [3]:
file_path1 = 'Electric Vehicle Population.xml'
file_path2= 'Green House gases data.xml'

In [4]:
df1 = read_xml_to_df(file_path1)
df2 = read_xml_to_df(file_path2)

In [5]:
# function to insert the data into mongo db
def insert_to_mongoDB(url, dataframe, database, collection):

    client = MongoClient(url)

    db = client[database]
    collection = db[collection]

    # Convert DataFrame to JSON
    json_data = dataframe.to_json(orient='records')

    # Insert Data into MongoDB
    data = json.loads(json_data)
    result = collection.insert_many(data)

    # Print confirmation
    if result.inserted_ids:
        print(f"Data inserted successfully")
    else:
        print("Failed to insert data")



In [6]:
mongo_url = 'mongodb://localhost:27017/'
database = 'Washington_State_Department_of_Licensing'
collection1 = 'Electric_Vehicle_Population'
collection2 = 'Green_House_Gases_Emission'
insert_to_mongoDB(mongo_url, df1, database, collection1)
insert_to_mongoDB(mongo_url, df2, database, collection2)

Data inserted successfully
Data inserted successfully
